##### Graph Coloring

* There a N locations, each with a name
* There are k colors
* There is a set of adjacency relationships between the locations
* The goal is to assign a color to each locations such that no two adjacent locations have the same color


### Task
Write the map color function as above.  It calls A Star Search.  Its output
is a dictionary assigning colors to locations, for example
```
{'WA': 'red', 'Q': "blue", ...}
```

Hint:  
* A state is a (partial) assignment of colors to locations
* To "expand" a state,
  * First choose some unassigned location to assign
  * The successors are all possible assignments of colors to that location
  
* For example, we start with an empty assignment {}.
* Then we choose 'WA' as a location to work on next
* The successors to this initial state would be 
  * {'WA': 'red'}
  * {'WA': 'green'}
  * {'WA': 'blue'}
  
The goal / solution checker needs to check two things:
* Every location must have a color assigned
* No two adjacent locations may be assigned the same color

In [126]:
from searchClientInterface import WorldState

# The successor function needs to make a deep copy of itself
import copy

class MapColorWorldState(WorldState):

    # Internal state is just the 'value' attribute
    def __init__(self, locations, colors, adjacencies):
        self._locations = locations
        self._colors = colors
        self._adjacencies = adjacencies
        self._assignments = {}
    
    # Convenience function to make counter objects print nicely
    # when they appear in search diagnostic messages
    def __str__(self):
        return "{" + str(self._assignments) + "}"
    
    #  These two methods are REQUIRED to make cycle checking work in the search
    #  Notice they depend on the object's internal state, so they must
    #  be customized to each new kind of WorldState
    
    def __eq__(self, other):
        if isinstance(other, MapColorWorldState):
            return self._assignments == other._assignments
        else:
            return False

    def __hash__(self):
        return hash(str(self._assignments))
    
    def successors(self):
        nextLocation = self.chooseUnassignedLocation()
        if nextLocation == None:
            return []
        return self.allAssignmentsFor(nextLocation)
    
    def chooseUnassignedLocation(self):
        for loc in self._locations:
            if not (loc in self._assignments):
                return loc
        return None
    
    def allAssignmentsFor(self, location):
        candidates = [(self.assignColor(location, c), (location, c)) for c in self._colors]
        return [c for c in candidates if c]
    
    def assignColor(self, location, color):
        s = copy.deepcopy(self)
        s._assignments[location] = color
        return s
   
    def isSolution(self):
        return self.isComplete() and self.isCorrect()
    
    def isComplete(self):
        return set(self._assignments.keys()) == set(self._locations)
        
    def isCorrect(self):
        print("iscorrect")
        for adj in self._adjacencies:
            a1, a2 = adj
            print(f"{a1} {a2}")
            if a1 in self._assignments and a2 in self._assignments and self._assignments[a1] == self._assignments[a2]:
                print(f"Failing on {self} because {a1} {a2} {self._assignments[a1]} {self._assignments[a2]}")
                return False
        return True

In [127]:
from searchClientInterface import Problem

# This class describes a problem by implementing initial() and
# isGoal(state)

class MapColorProblem(Problem):
    def __init__(self, locations, colors, adjacencies):
        self._state = MapColorWorldState(locations, colors, adjacencies)
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state.isSolution()

In [128]:
from searchClientInterface import BFSEvaluator
from searchFramework import aStarSearch

def mapColor(l, c, a):
    return aStarSearch(MapColorProblem(l, c, a), BFSEvaluator(), verbose=1000, limit=None)

In [129]:
#  Here is the example from Russel and Norvig, Figure 6.1
locations = ['WA', 'NT', 'Q', 'SA', 'NSW', 'V', 'T']
colors = ["red", "green", "blue"]
adjacencies = [('WA', 'NT'), ('WA', 'SA'), ('NT', 'SA'), ('NT', 'Q'), 
               ('SA', 'Q'), ('Q', 'NSW'), ('SA', 'NSW'), ('SA', 'V'),
               ('NSW', 'V')]

mapColor(locations, colors, adjacencies)

Visited 1000 world is {{'WA': 'blue', 'NT': 'green', 'Q': 'blue', 'SA': 'green', 'NSW': 'green', 'V': 'blue'}}
Skipped 0 Fringe is size 1998
Evaluation is 6 with actions 6
iscorrect
WA NT
Failing on {{'WA': 'red', 'NT': 'red', 'Q': 'red', 'SA': 'red', 'NSW': 'red', 'V': 'red', 'T': 'red'}} because WA NT red red
iscorrect
WA NT
Failing on {{'WA': 'red', 'NT': 'red', 'Q': 'red', 'SA': 'red', 'NSW': 'red', 'V': 'red', 'T': 'green'}} because WA NT red red
iscorrect
WA NT
Failing on {{'WA': 'red', 'NT': 'red', 'Q': 'red', 'SA': 'red', 'NSW': 'red', 'V': 'red', 'T': 'blue'}} because WA NT red red
iscorrect
WA NT
Failing on {{'WA': 'red', 'NT': 'red', 'Q': 'red', 'SA': 'red', 'NSW': 'red', 'V': 'green', 'T': 'red'}} because WA NT red red
iscorrect
WA NT
Failing on {{'WA': 'red', 'NT': 'red', 'Q': 'red', 'SA': 'red', 'NSW': 'red', 'V': 'green', 'T': 'green'}} because WA NT red red
iscorrect
WA NT
Failing on {{'WA': 'red', 'NT': 'red', 'Q': 'red', 'SA': 'red', 'NSW': 'red', 'V': 'green', 'T': '

([('WA', 'red'),
  ('NT', 'green'),
  ('Q', 'red'),
  ('SA', 'blue'),
  ('NSW', 'green'),
  ('V', 'red'),
  ('T', 'red')],
 (0.53125, 1400, 0, 2187))

#### One More Task

Write a function 
```
minRequiredColors(locations, adjacencies)
```
that returns the minimum number of colors required to color the graph specified by locations and adjacencies.
What is the minimum coloring for the Australia graph?
